# Notes for PyPot

1.    We can access registers to control position, speed or torque of any motor. Basically through Dynamixel low-level IO or Controller API .
2.    Port can only be accessed by a single DxlIO instance. Not quite sure about how this means, but seems to have something to do with restart the kernel every time.
3.    Communication is thread-safe.

# Things about DxlIO instance

use get_available_ports() to auto-discover compatible devices

In [1]:
 

import pypot.dynamixel

ports = pypot.dynamixel.get_available_ports()

if not ports:

    raise IOError('no port found')

print('ports found', ports)

dxl_io = pypot.dynamixel.DxlIO(ports[...]) # set a DxlIO instance

IOError: no port found

Your pypot io's baud rate need to be the same as the motors configuration. By default, the former is 1000000 and the latter is 57140. This can be done like this. 

To detect the motors and find their id you can scan the bus. To avoid spending a long time searching all possible values, you can add a list of values to test:

In [ ]:
 

dxl_io = pypot.dynamixel.DxlIO(port, baudrate=57600)

dxl_io.scan() #This might take a while

dxl_io.scan([0,1,2,3,4,5,....])

Once id of motors connected, we can begin to access their functions by using their id. The position are gained in degrees. 

In [ ]:
dxl_io.get_present_position((4, )) #retrieve position of id 4

dxl_io.set_goal_position({4: 0}) #set the id 4 to 0 degree

* For a DxlIO instance, all getter function takes a list as input, while all setter function takes a dictionary as input. 

For the MX motors, end points are 180 and -180 degrees. For AX and RX motors, values are 150 and -150 degrees. ###????????????

# Things about synchronized control

Using class Robot to do higher level control. Low-level control is slower while this faster control enables sychronization loop. First define robot using a configuration file (like a json file). Then set up synchronization loops.

. More precisely, through the use of the class Robot you can:

   * automatically initialize all connections (make transparent the use of multiple USB2serial connections),
   * define offset and direct attributes for motors,
   * automatically define accessor for motors and their most frequently used registers (such as goal_position, present_speed, present_load, pid, compliant),
   * define read/write synchronization loop that will run in background.


##  Configuration
Configuration file is like a python dictionary contains important fields like controllers motors and motorgroups. 

__controllers__ - This key holds the information pertaining to a controller and all the items connected to its bus.

__motors__ - This is a description of all the custom setup values for each motor. Meta information, such as the motor access name or orientation, is also included here. It is also there that you will set the angle limits of the motor.

__motorgroups__ - This is used to define alias of a group of motors (e.g. left_leg).


* The example config files don't have one for poppy_humanoid. We should create one! 
One way to generate config file is simply make a dict in python.
* the example provided with pypot and modify it (e.g. pypot.robot.config.ergo_robot_config):

For __controllers__, an example is like this:
 For each of them, you should indicate whether or not to use the SYNC_READ instruction (only the USB2AX device currently supported it). When you describe your controller, you must also include the port that the device is connected to (see Opening/Closing a communication port). In this section, you can also specify which robotis protocol to use (if not specified it uses the v1). You also have to specify which motors are attached to this bus. You can either give individual motors or groups (see the sections below):

In [2]:
 

my_config['controllers'] = {}

my_config['controllers']['upper_body_controller'] = {

    'port': '/dev/ttyUSB0', # set the port the device is connected to, can be 'auto'

    'sync_read': False, # when using SYNC_READ, can access motors synchronizlly

    'attached_motors': ['torso', 'head', 'arms'], # specify which motors attach to this bus.

    'protocol':1,

}

NameError: name 'my_config' is not defined

__motorgroups__: Here, you can define the different motors group corresponding to the structure of your robot. It will automatically create an alias for the group. Groups can be nested, i.e. a group can be included inside another group, as in the example below:

In [ ]:
my_config['motorgroups'] = {
    'torso': ['arms', 'head_x', 'head_y'],
    'arms': ['left_arm', 'right_arm'],
    'left_arm': ['l_shoulder_x', 'l_shoulder_y', 'l_elbow'],
    'right_arm': ['r_shoulder_x', 'r_shoulder_y', 'r_elbow']
}

__motors__: Then, you add all the motors. The attributes are not optional and describe how the motors can be used in the software. You have to specify the type of motor, it will change which attributes are available (e.g. compliance margin versus pid gains). The name and id are used to access the motor specifically. Orientation describes whether the motor will act in an anti-clockwise fashion (direct) or clockwise (indirect). You should also provide the angle limits of your motor. They will be checked automatically at every start up and changed if needed:

In [ ]:
 

# Add all the motors, and these attributes are not optional

my_config['motors'] = {}

my_config['motors']['l_hip_y'] = {

    'id':11, # name and id specification

    'type': 'MX-28', # see more in document

    'orientation': 'direct', # direct is anti-clockwise, indirect is clockwise

    'offset': 0.0, # see more in document

    'angle_limit': (-90.0, 90.0), will be changed automatically if needed

}

Use from_config() function to read configuration dict you've created. 

In [ ]:
import pypot.robot

robot = pypot.robot.from_config(my_config)

for m in robot.left_arm:
    print(m.present_position)

__Robot__ can also be created automatically. But is quite slow. This way is usually used to discover the robot configuration and then export it for later reuse. 

If you have manually created your Robot (or thanks to the __autodetect_robot()__ function), you can then use the __to_config()__ method to export the Robot current configuration.

In [ ]:
from pypot.dynamixel import autodetect_robot

my_robot = autodetect_robot()

​

import json

config = my_robot.to_config()

with open('my_robot.json', 'wb') as f:

    json.dump(config, f)

# simply re-create robot using .json file

from pypot.robot import from_json

new_robot = from_json('my_robot.json')

# Control
Things about control motors can be accessed via DxlMotor. This is very important!!!

Once __Robot__ is created, all things is done and synchronization automatically starts.

* Read present position, speed, load at 50Hz
* writes the goal position, moving speed and torque limit at 50Hz
* writes the pid or compliance margin/slope at 10Hz

# Close the robot after done
Make sure everything gets cleaned correctly after done. Always call the __close()__ method! Use __contextlib.closing()__ decorator to make sure close() method is always called since something might prevent this method. 

In [ ]:
 

from contextlib import closing

import pypot.robot

​

# The closing decorator make sure that the close function will be called

# on the object passed as argument when the with block is exited.

with closing(pypot.robot.from_json('myconfig.json')) as my_robot:

    # do stuff without having to make sure not to forget to close my_robot!

    pass